# Econ 1923 - Product Differentiation Project

## 1 Preparation
### 1.1 Obtaining the data
Create a data set containing nutrition information about cereal boxes. You can use all sources including the nutrition APIs, stores web pages, actual grocery stores, your pantry, etc.
At the end of this process, you should have a dataframe that contains as many cereal boxes as possible and for each, you should have several nutritional facts and (ideally) serving sizes (cups or grams).
The result of this part can be several dataframes, each resulting from the different data sources that you used.

Sources: Edemam API, GitHub JSON Files, personal cereals


In [8]:
#Importing packages
import requests, json, time
import pandas as pd
from matplotlib import pyplot as plt
%run ../APIkeys.py

In [9]:
app_id = os.environ['EDAMAM_API_id']
key = os.environ['EDAMAM_API_key']
baseURL = 'https://api.edamam.com/api/food-database/v2/parser?app_id='+app_id+'&app_key='+key+'&upc='
endURL = '&nutrition-type=cooking&category=packaged-foods'

pd.set_option('display.max_rows', None, 'display.max_columns', None)

In [10]:
def checkUPC_V2(upc):
    app_id = os.environ['EDAMAM_API_id']
    key = os.environ['EDAMAM_API_key']
    baseURL = 'https://api.edamam.com/api/food-database/v2/parser?app_id='+app_id+'&app_key='+key+'&upc='
    endURL = '&nutrition-type=cooking&category=packaged-foods'
    url = baseURL + upc + endURL
    info = requests.get(url).json()
    if 'error' in info:
        return info
    else:
        answer = info['hints'][0]['food']['nutrients']
        
        sizeList = info['hints'][0]['food']['servingSizes']
        n = len(sizeList)
        for i in range(n):
            if sizeList[i]['label'] == 'Gram':
                answer['servingSize_gram'] = info['hints'][0]['food']['servingSizes'][i]['quantity']
            if sizeList[i]['label'] == 'Cup':
                answer['servingSize_cup'] = info['hints'][0]['food']['servingSizes'][i]['quantity']
        
        answer['label'] = info['hints'][0]['food']['label']
        return answer

In [11]:
#We add a 3 second interval between each call to stay under the limit for the Edamam API.

def multiUPC(shelf):
    a = []
    missing = []
    for upc in shelf:
        info = checkUPC_V2(upc)
        if 'error' in info:
            print('UPC ',upc, ' does not exist in the API\'s data base')
            missing.append(upc)
            time.sleep(3)
        else:
            print('UPC ',upc, ' exists in the API\'s data base')
            a.append(info)
            time.sleep(3)
    return a

In [12]:
#reading excel and csv files containing states' WIC 
#Approved Products Lists (APLs) and filtering down to cereal UPCs

df_IL = pd.read_excel('IL_WIC_APL.xlsx') #Illinois
df_IL = df_IL.drop([174], axis=0)
df_IL = df_IL.reset_index(drop=True)

df_MA = pd.read_excel('MA_WIC_APL.xlsx')
df_MA = df_MA[df_MA['Category Name']=='Breakfast Cereal'] #Massachusetts
df_MA = df_MA.reset_index(drop=True)

df_MN = pd.read_excel('MN_WIC_APL.xlsx')#Minnesota

df_NE = pd.read_excel('NE_WIC_APL.xlsx')#Nebraska

df_NV = pd.read_csv('NV_WIC_APL.csv')#Nevada

df_NY = pd.read_excel('NY_WIC_APL.xlsx')#New York S
df_NY = df_NY[df_NY['Category']=='Breakfast Cereal ']
df_NY = df_NY.reset_index(drop=True)

In [16]:
#Isolating UPCs into string arrays and combining
IL_UPCs = []
for i in range(0,len(df_IL['UPC/PLU Code'])-1):
    IL_UPCs.append(df_IL['UPC/PLU Code'][i][7:18])
    
MA_UPCs = []
for i in range(0,len(df_MA['UPC'])-1):
    MA_UPCs.append(str(df_MA['UPC'][i]))

MN_UPCs = []
for i in range(0,len(df_MN['UPC_PLU'])-1):
    MN_UPCs.append(str(df_MN['UPC_PLU'][i]))
    
NE_UPCs = []
for i in range(0,len(df_NE['UPC'])-1):
    NE_UPCs.append(str(df_NE['UPC'][i]))

NV_UPCs = []
for i in range(0,len(df_NV['UPC'])-1):
    if len(df_NV['UPC'][i])==14:
        NV_UPCs.append(df_NV['UPC'][i][2:13])
    elif len(df_NV['UPC'][i])==15:
        NV_UPCs.append(df_NV['UPC'][i][3:14])

NY_UPCs = []
for i in range(0,len(df_NY['UPC/PLU Number'])-1):
    if len(str(df_NY['UPC/PLU Number'][i]))==11:
        NY_UPCs.append(str(df_NY['UPC/PLU Number'][i]))
    elif len(str(df_NY['UPC/PLU Number'][i]))==12:
        NY_UPCs.append(str(df_NY['UPC/PLU Number'][i])[1:12])
        
UPCs = IL_UPCs + MA_UPCs + MN_UPCs + NE_UPCs + NV_UPCs + NY_UPCs #combining
UPCs = list(set(UPCs)) #removes duplicate UPCs, and then coerces into a list

In [19]:
for upc in UPCs:
    print(UPCs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




['11110851970', '84912014252', '75450152425', '38000281686', '16000487710', '884912359162', '38000269967', '88267169762', '11110878298', '42400181545', '41270830300', '21859046032', '36800110175', '16000176348', '21130280490', '11161029175', '36800448643', '688267169663', '91669300388', '73296034516', '70784472355', '688267078736', '38000214691', '16000171022', '77890449110', '42187408903', '41735088628', '11161159599', '16000487987', '41190054008', '41735088635', '11110853301', '73296313604', '16000487697', '17544135470', '75130414089', '11110861641', '20794012293', '11161020318', '16000275058', '36800317581', '16000178182', '21333061056', '75130404059', '41190022687', '46567007939', '36800421745', '38000300011', '41735050106', '30000061190', '41512082931', '75436999068', '41190054022', '38000027208', '884912320445', '41268164226', '78742121703', '42187408279', '36800110113', '51933347588', '84912249272', '15400874434', '18894110286', '41497538485', '11150301909', '84912320490', '884

In [7]:
boxes, missing = multiUPC(UPCs)

####### This is is for if the above function doesn't work, for some reason it is buggy when running large batches, but running it manually without the funtion works for some reason

####### Change this block to Code to run

boxes = []
missing = []
for upc in UPCs:
    info = checkUPC_V2(upc)
    if 'error' in info:
        print('UPC ',upc, ' does not exist in the API\'s data base')
        missing.append(upc)
        time.sleep(3)
    else:
        print('UPC ',upc, ' exists in the API\'s data base')
        boxes.append(info)
        time.sleep(3)

In [10]:
Cereals = pd.DataFrame(boxes)

In [21]:
#The dataframe appears to still have some duplicate cereal labels, so we're dropping them
Cereals = Cereals.drop_duplicates(subset=['label'],keep='last')

#Add source column and fill with 
Cereals['Source'] = 'EDAMAM_API'

#Since calling all the UPCs takes a long time, we're saving the 
#dataframe into an excel sheet for quicker loader later on
Cereals.to_excel("Cereals.xlsx", sheet_name='Cereal Data')

In [23]:
Cereals = pd.read_excel('Cereals.xlsx')
len(Cereals['label'])

Cereals

,Unnamed: 0,Unnamed: 0.1,ENERC_KCAL,FAT,FASAT,FATRN,CHOCDF,FIBTG,SUGAR,PROCNT,CHOLE,NA,CA,K,FE,ZN,VITC,THIA,RIBF,NIA,VITB6A,FOLAC,VITB12,VITD,servingSize_cup,servingSize_gram,label,SUGAR.added,P,MG,FAMS,FAPU,TOCPHA,VITK1,Sugar.alcohol,WATER,Source
0,0,0,387.096774,0.000000,0.000000,0.0,87.096774,3.225806,9.677419,6.451613,0.000000,709.677419,322.580645,112.903226,29.032258,12.096774,19.354839,1.225806,1.387097,16.129032,1.612903,645.161290,4.838710,3.225806,1.000000,31.000000,"Fareway Cereal Corn Squares, 12.0 Oz",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EDAMAM_API
1,1,2,357.142857,7.142857,0.000000,0.0,71.428571,10.714286,3.571429,10.714286,0.000000,500.000000,357.142857,607.142857,28.928573,13.392857,21.428571,1.357143,1.535714,17.857143,1.785714,714.285714,5.357143,3.571429,1.000000,28.000000,"Cheerios Cereal Double Pack, 2.0 Ea",NaN,357.142857,142.857143,NaN,NaN,NaN,NaN,NaN,NaN,EDAMAM_API
2,2,3,364.000000,0.000000,0.000000,0.0,75.760002,3.000000,3.030000,12.120000,0.000000,258.000000,606.000000,NaN,27.270000,NaN,0.000000,0.000000,0.206000,3.636000,NaN,NaN,NaN,NaN,NaN,33.000000,Creamy Wheat Enriched Farina,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,EDAMAM_API
3,3,4,363.762727,3.306934,0.000000,0.0,76.059479,9.920802,19.841603,6.613868,0.000000,396.832066,330.693388,330.693388,26.786166,12.401002,19.841603,1.256635,1.421982,16.534669,1.653467,661.386777,4.960401,8.267335,1.000000,NaN,Ahold Toasted Cereal Oats & o's Multigrain,NaN,264.554711,79.366413,0.000000,1.653467,NaN,NaN,NaN,NaN,EDAMAM_API
4,4,5,349.800123,2.057648,0.000000,0.0,78.190616,10.288239,6.172943,12.345887,0.000000,432.106034,NaN,390.953078,29.630127,4.629708,NaN,0.781906,0.884789,10.288239,1.028824,205.764778,3.086472,2.057648,0.500000,NaN,"Hannaford Nutty Nuggets Wheat & Barley Cereal,...",NaN,308.647167,82.305911,NaN,NaN,NaN,NaN,NaN,NaN,EDAMAM_API
5,5,6,387.000000,0.000000,0.000000,0.0,87.099998,3.200000,9.680000,6.450000,0.000000,710.000000,323.000000,113.000000,29.030001,12.100000,19.400000,1.000000,1.371000,16.129000,1.613000,NaN,4.840000,NaN,1.000000,31.000000,Corn Biscuits Cereal,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,EDAMAM_API
6,6,8,411.124108,7.906233,0.000000,0.0,79.062329,6.324986,18.974959,6.324986,0.000000,442.749040,NaN,237.186986,34.154927,0.948748,NaN,1.201747,1.359872,15.812466,1.581247,NaN,4.743740,3.162493,0.750000,32.000000,"Honey Bunches of Oats Cereal With Almonds, 14....",NaN,189.749588,50.599890,NaN,NaN,NaN,NaN,NaN,NaN,EDAMAM_API
7,7,9,357.142857,5.357143,0.000000,0.0,64.285714,10.714286,3.571429,14.285714,0.000000,285.714286,357.142857,0.000000,28.928573,0.000000,0.000000,1.071429,1.214286,14.285714,1.428571,285.714286,0.000000,0.000000,NaN,28.000000,"Price Chopper Instant Oatmeal, Regular Flavor,...",NaN,357.142857,142.857143,1.785714,1.785714,0.000000,0.0,0.0,NaN,EDAMAM_API
8,8,12,383.000000,5.830000,0.000000,0.0,63.330002,6.700000,15.000000,25.000000,0.000000,450.000000,217.000000,267.000000,30.000000,NaN,NaN,NaN,0.567000,6.667000,0.667000,192.000000,NaN,NaN,NaN,60.000000,"Protein Honey Almond Ancient Grains Cereal, Ho...",15.000000,167.000000,67.000000,2.500000,1.670000,NaN,NaN,NaN,NaN,EDAMAM_API
9,9,13,357.142857,5.357143,0.000000,0.0,75.000000,10.714286,3.571429,10.714286,0.000000,571.428571,357.142857,607.142857,32.142857,13.392857,21.428571,1.357143,1.535714,17.857143,1.785714,714.285714,5.357143,7.142857,1.000000,28.000000,"Fareway Cereal , 12.0 Oz",NaN,535.714286,142.857143,NaN,NaN,NaN,NaN,NaN,NaN,EDAMAM_API


### 1.2 Cleaning and re-scaling
Various data sources may contain different variable names, different units, or different serving sizes. Also, missing values can be an issue. Here, you are asked to:
- Make sure that variable names in all dataframes that you created match.
- Make sure that all nutritional measurements in all data frames are the same (e.g. per cup or per 100gr).
- Merge the different data sources.
 
The final data set should include a column indicating the source of the observation (i.e. is it from the API, entered manually from a certain store, etc.).

### 1.3 Descriptive statistics
Create a table with descriptive statistics (Mean, Median, standard deviation, range, number of missing observations, etc.

## 2 Market Analysis
### 2.1 Visualisation
Create scatter plots of the data. Here you have to try several pairs of characteristics on which it seems like the products are more differentiated. This is a visual exercise but you can support your claims by looking at standard deviations and correlations.

### 2.2 K-Means clustering
Using the K-means clustering technique analyze the degree of product differentiation. Based on descriptive statistics, data availability, and scatter plots, choose pairs of features (i.e., nutrition facts) on which you believe the sellers differentiate themselves. Quantify your answer. You should try different K parameters. Report the goodness of fit (total sum of errors). *Instead of different pairs of features you can focus on the same pair and compare different locations or different supermarkets.*

## 3 Conclusions
Summarize your findings and draw conclusions from them. What can you recommend to cereal producers on the market? Do you find this market to be saturated? Here, I leave it to you to analyze your findings as you see fit. Given all the effort you have put, This section should be as very important.